In [183]:
import autograd.numpy as np
from autograd import grad, elementwise_grad, jacobian
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn import datasets
import time


def feedforward(x, P, act_func):
    dim = len(P)
    z = np.array(x)
    
    for i in range(dim):
        z = np.concatenate((z, np.ones((z.shape[0], 1))), axis=1)
        z = act_func[i](z @ P[i])

    return z


def generate_P(dim):
    n = len(dim) - 1
    P = n * [None]
    for i in range(n):
        P[i] = np.random.normal(0, 1, (dim[i] + 1, dim[i + 1]))

    return P

In [207]:
A = np.array([[3, 2, 4], [2, 0, 2], [4, 2, 3]])
x0 = np.array([1, 0, 0])#np.random.normal(0, 1, 3)

def g_trial(t, P, act_func):
    return np.exp(-t)*x0 +  (1 - np.exp(-t))*feedforward(t, P, act_func)


g_trial_t = jacobian(g_trial)


def f(A, x):
    x = x[:, np.newaxis]
    return (x.T@x)*A@x - (x.T@A@x)*x 

    

def cost(t, P, act_func):
    x = g_trial(t, P, act_func)
    
    LHS = g_trial_t(t, P, act_func)
    LHS = np.array([LHS[i,:,i].flatten() for i in range(len(t))])
    print(x)
    RHS = np.array([f(A,i) for i in x])
    
    print(RHS[:,:,0])
    print(LHS)
    
    return np.mean((LHS - RHS[:,:,0])**2)
    

grad = elementwise_grad(cost,1)

In [209]:

np.random.seed(40)
#%matplotlib notebook

t = np.expand_dims(np.linspace(0, 10, 20), axis = -1)



P = generate_P((1, 100, 50, 3))
act_func = [np.tanh, np.tanh, lambda x: x]

N = 1000

print(grad(t, P, act_func)[0])


Autograd ArrayBox with value [[ 1.          0.          0.        ]
 [ 0.51721653 -0.70421775  1.04658775]
 [ 3.77520574 -3.88118923  2.98738039]
 [ 4.94908172 -4.52416976  1.38362598]
 [ 3.15397986 -3.91245546 -1.93230088]
 [ 2.42495914 -5.0675084  -2.98212166]
 [ 1.04702407 -5.36134863 -3.32727839]
 [ 0.5443869  -5.62573958 -3.93999201]
 [ 0.81944182 -6.03744172 -4.28246057]
 [ 1.08245907 -6.31425022 -4.40079636]
 [ 1.24475036 -6.50859121 -4.46796313]
 [ 1.38010776 -6.68828749 -4.5350034 ]
 [ 1.52969816 -6.88339247 -4.61992732]
 [ 1.6922146  -7.09206442 -4.73630207]
 [ 1.8488304  -7.29698382 -4.89280794]
 [ 1.9846703  -7.47985595 -5.09039352]
 [ 2.09486031 -7.6276463  -5.32455468]
 [ 2.18196385 -7.7339412  -5.58916128]
 [ 2.25159582 -7.79891434 -5.87861003]
 [ 2.3086286  -7.82926741 -6.1883753 ]]
Autograd ArrayBox with value [[ 0.00000000e+00  2.00000000e+00  4.00000000e+00]
 [ 5.97158683e+00  8.64030024e+00  2.86268334e+00]
 [ 3.86458313e+02  7.29763792e+02  4.59731120e+02]
 [ 4.352

In [198]:
"""
for i in range(N):
    if i % (N / 100) == 0:
        print(i / (N / 100))
        print(cost(t, P, act_func))
        print("-------------")
   
    gradient = grad(t, P, act_func)

    P[0] -= 0.000001 * gradient[0]
    P[1] -= 0.000001 * gradient[1]
    P[2] -= 0.000001 * gradient[2]

print(cost(t, P, act_func))

v = g_trial(np.array([[10]]), P, act_func)
print(v.shape)
print(v@A@v.T/(v@v.T))
"""
"""
for i in range(N):
    if i % (N / 100) == 0:
        print(i / (N / 100))
        print(cost(t, P, act_func))
        print("-------------")
   
    gradient = grad(t, P, act_func)

    P[0] -= 0.0001 * gradient[0]
    P[1] -= 0.0001 * gradient[1]

print(cost(x, P, act_func))
"""


"""
a = 0

g1 = g2 = g3

fig, ax = plt.subplots()
plot, = ax.plot(x, 10*np.exp(-2*x))
ax.plot(x, 10*np.exp(-2*x))

texts = [fig.text(0.5, 0.5 - i * 0.05, '') for i in range(3)]

for i in range(N):
    if i % (N / 100) == 0:
        print(i / (N / 100))
        print(cost(x, P, act_func))
        print("-------------")
    plot.set_ydata(g_trial(x, P, act_func))
   

    gradient = grad(x, P, act_func)
    g1 = gradient[0] + a * g1
    g2 = gradient[1] + a * g2
    g3 = gradient[2] + a * g3
    
    values = [f"{np.mean(np.abs(g1))}", f"{np.mean(np.abs(g2))}", f"{np.mean(np.abs(g3))}"]    
    [txt.set_text(str(val)) for txt, val in zip(texts, values)]
    fig.canvas.draw()

    P[0] -= 0.0001 * g1
    P[1] -= 0.0001 * g2
    P[2] -= 0.0001 * g3

print(cost(x, P, act_func))
"""

'\na = 0\n\ng1 = g2 = g3\n\nfig, ax = plt.subplots()\nplot, = ax.plot(x, 10*np.exp(-2*x))\nax.plot(x, 10*np.exp(-2*x))\n\ntexts = [fig.text(0.5, 0.5 - i * 0.05, \'\') for i in range(3)]\n\nfor i in range(N):\n    if i % (N / 100) == 0:\n        print(i / (N / 100))\n        print(cost(x, P, act_func))\n        print("-------------")\n    plot.set_ydata(g_trial(x, P, act_func))\n   \n\n    gradient = grad(x, P, act_func)\n    g1 = gradient[0] + a * g1\n    g2 = gradient[1] + a * g2\n    g3 = gradient[2] + a * g3\n    \n    values = [f"{np.mean(np.abs(g1))}", f"{np.mean(np.abs(g2))}", f"{np.mean(np.abs(g3))}"]    \n    [txt.set_text(str(val)) for txt, val in zip(texts, values)]\n    fig.canvas.draw()\n\n    P[0] -= 0.0001 * g1\n    P[1] -= 0.0001 * g2\n    P[2] -= 0.0001 * g3\n\nprint(cost(x, P, act_func))\n'

In [42]:
def func(t):
    return np.array([np.sin(t), np.cos(t)])

dfunc = jacobian(func, 0)

t = np.array([0.2, 0.3])
print(dfunc(t))

[[[ 0.98006658  0.        ]
  [ 0.          0.95533649]]

 [[-0.19866933  0.        ]
  [ 0.         -0.29552021]]]
